In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import threading
import winsound  # Importing winsound for the alarm sound

# Define alarm frequency and duration
ALARM_FREQUENCY = 1000  # Frequency in Hz (1000 Hz is a typical alarm sound)
ALARM_DURATION = 2000  # Duration in milliseconds (2000ms = 2 seconds)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Capture video from the webcam
cap = cv2.VideoCapture(0)

# EAR Calculation Function
def compute(ptA, ptB):
    return np.linalg.norm(np.array(ptA) - np.array(ptB))

def blinked(landmarks):
    """Calculate EAR (Eye Aspect Ratio) using MediaPipe landmarks."""
    LEFT_EYE = [33, 160, 158, 133, 153, 144]
    RIGHT_EYE = [362, 385, 387, 263, 373, 380]

    left_eye = np.array([landmarks[i] for i in LEFT_EYE])
    right_eye = np.array([landmarks[i] for i in RIGHT_EYE])

    left_ear = (compute(left_eye[1], left_eye[5]) + compute(left_eye[2], left_eye[4])) / (2.0 * compute(left_eye[0], left_eye[3]))
    right_ear = (compute(right_eye[1], right_eye[5]) + compute(right_eye[2], right_eye[4])) / (2.0 * compute(right_eye[0], right_eye[3]))

    return (left_ear + right_ear) / 2.0

# EAR Thresholds
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 6

# Status tracking
sleep, drowsy, active = 0, 0, 0
status, color = "", (0, 0, 0)
alarm_playing = False

# Function to play alarm using winsound
def play_alarm():
    winsound.Beep(ALARM_FREQUENCY, ALARM_DURATION)  # Beep for the specified frequency and duration

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = {i: (lm.x * frame.shape[1], lm.y * frame.shape[0]) for i, lm in enumerate(face_landmarks.landmark)}

            ear = blinked(landmarks)

            print(f"EAR: {ear:.3f}")  # Debugging EAR values

            if ear < EAR_THRESHOLD:
                sleep += 1
                drowsy, active = 0, 0
                if sleep >= EAR_CONSEC_FRAMES:
                    status, color = "SLEEPING !!!", (255, 0, 0)
                    threading.Thread(target=play_alarm, daemon=True).start()
            elif EAR_THRESHOLD <= ear < EAR_THRESHOLD + 0.04:
                drowsy += 1
                sleep, active = 0, 0
                if drowsy >= EAR_CONSEC_FRAMES:
                    status, color = "Drowsy !", (0, 0, 255)
                    threading.Thread(target=play_alarm, daemon=True).start()
            else:
                active += 1
                sleep, drowsy = 0, 0
                status, color = "Active :)", (0, 255, 0)
                alarm_playing = False  # Reset alarm status when active

            cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

            for _, (x, y) in landmarks.items():
                cv2.circle(frame, (int(x), int(y)), 1, (0, 255, 0), -1)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


EAR: 0.292
EAR: 0.312
EAR: 0.294
EAR: 0.299
EAR: 0.283
EAR: 0.282
EAR: 0.285
EAR: 0.288
EAR: 0.284
EAR: 0.285
EAR: 0.281
EAR: 0.279
EAR: 0.272
EAR: 0.271
EAR: 0.279
EAR: 0.285
EAR: 0.274
EAR: 0.272
EAR: 0.272
EAR: 0.277
EAR: 0.268
EAR: 0.274
EAR: 0.274
EAR: 0.256
EAR: 0.243
EAR: 0.228
EAR: 0.242
EAR: 0.240
EAR: 0.234
EAR: 0.234
EAR: 0.236
EAR: 0.236
EAR: 0.235
EAR: 0.227
EAR: 0.238
EAR: 0.233
EAR: 0.244
EAR: 0.243
EAR: 0.252
EAR: 0.240
EAR: 0.229
EAR: 0.231
EAR: 0.241
EAR: 0.254
EAR: 0.268
EAR: 0.277
EAR: 0.283
EAR: 0.285
EAR: 0.282
EAR: 0.285
EAR: 0.291
EAR: 0.287
EAR: 0.276
EAR: 0.278
EAR: 0.270
EAR: 0.272
EAR: 0.278
EAR: 0.277
EAR: 0.280
EAR: 0.278
EAR: 0.278
EAR: 0.291
EAR: 0.275
EAR: 0.283
EAR: 0.281
EAR: 0.280
EAR: 0.274
EAR: 0.271
EAR: 0.283
EAR: 0.288
EAR: 0.289
EAR: 0.285
EAR: 0.282
EAR: 0.275
EAR: 0.269
EAR: 0.281
EAR: 0.271
EAR: 0.272
EAR: 0.262
EAR: 0.261
EAR: 0.255
EAR: 0.247
EAR: 0.264
EAR: 0.249
EAR: 0.251
EAR: 0.256
EAR: 0.262
EAR: 0.262
EAR: 0.259
EAR: 0.252
EAR: 0.251